In [1]:
import pandas as pd
import string
import requests
import os


In [2]:
file_name = '/Users/caged/Desktop/Mining the disclosures/RSN-MiningDisclosures15-Universe_2-4.csv'
exclude = set(string.punctuation)
def clean_names(sub):
    sub['name'] = sub['Company Name'].str.upper()
    sub['name_clean'] = sub.name.apply(lambda s: ''.join(ch for ch in s if ch not in exclude))
    sub['exchange'] = sub['Ticker Symbol'].str.split(':').str[0]
    sub['ticker'] = sub['Ticker Symbol'].str.split(':').str[1]
    return sub
tickers = pd.read_csv(file_name, usecols=['Company Name', 'Ticker Symbol'], dtype='str')
tickers = clean_names(tickers)
tickers = tickers.dropna()
tickers.head()

,Company Name,Ticker Symbol,name,name_clean,exchange,ticker
0,3D SYSTEMS CORP,NYSE:DDD,3D SYSTEMS CORP,3D SYSTEMS CORP,NYSE,DDD
1,3M Co.,NYSE:MMM,3M CO.,3M CO,NYSE,MMM
2,"A10 Networks, Inc.",NYSE:ATEN,"A10 NETWORKS, INC.",A10 NETWORKS INC,NYSE,ATEN
3,AAON INC,NasdaqGS:AAON,AAON INC,AAON INC,NasdaqGS,AAON
4,AAR CORP,NYSE:AIR,AAR CORP,AAR CORP,NYSE,AIR


In [3]:
tickers.count()

Company Name     2581
Ticker Symbol    2581
name             2581
name_clean       2581
exchange         2581
ticker           2581
dtype: int64

In [4]:
unique_tickers = tickers.ticker.unique()
len(unique_tickers)

2519

In [5]:
def look_up_company_by_ticker(list_of_tickers):
    url = "http://edgaronline.api.mashery.com/v2/companies"
    params = {
        'appkey': os.environ.get('EDGAR_KEY'),
        'primarysymbols': ','.join(list_of_tickers),
    }
    response = requests.get(url=url, params=params)
    return response

In [9]:
# sample_1 = unique_tickers[0:50]
# sample_2 = unique_tickers[50:100]
list_of_samples = []
for x in range(250):
    sample = unique_tickers[x*10: (x+1)*10]
    list_of_samples.append(sample)
len(list_of_samples)

250

In [27]:
good_responses = []
bad_responses = []
for i, sample in enumerate(list_of_samples):
    print(i)
    response = look_up_company_by_ticker(sample)
    if response.status_code == 200:
        good_responses.append(response)
    else:
        bad_responses.append(response)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [28]:
len(good_responses)

66

In [29]:
len(bad_responses)

184

In [31]:
bad_responses[0].status_code

400

In [38]:
cleaned_ciks = pd.DataFrame()
for response in good_responses:
    if response.status_code == 200:
        for row in response.json()['result']['rows']:
            fields = {}
            for field in row['values']:
                fields[field['field']] = field['value']
            cleaned_ciks = pd.concat([cleaned_ciks, pd.DataFrame([fields])], ignore_index=True)
cleaned_ciks.tail()  

,cik,companyname,entityid,primaryexchange,primarysymbol,siccode,sicdescription
554,0001555365,WHITEWAVE FOODS CO,887169,NYSE,WWAV,2020,Dairy Products
555,0000108312,"WOODWARD, INC.",11583,Nasdaq Global Market,WWD,3620,Electrical Industrial Apparatus
556,0001091907,WORLD WRESTLING ENTERTAINMENTINC,7520,NYSE,WWE,7812,Motion Picture and Video Tape Production
557,0000110471,WOLVERINE WORLD WIDE INC /DE/,335,NYSE,WWW,3140,"Footwear, Except Rubber"
558,0000106535,WEYERHAEUSER CO,4423,NYSE,WY,6798,Real Estate Investment Trusts


In [39]:
cleaned_ciks.to_csv('clean_ciks_partial.csv')